# README

This is a copy of the funtionality in `bpop_optimize_stn.py` in notebook format.

## HOWTO: Parallel Execution

To run parallel optimization (individuals in population evaluated over many cores), the ipyprallel module can be used. To use ipyparallel you must start a controller and a number of engine (worker) instances before starting ipython (see http://ipyparallel.readthedocs.io/en/latest/intro.html), e.g:

```bash
ipcluster start -n 6
ipengine --debug # only if you want to start a kernel with visible output
```

, where the -n argument is the number of workers / cores. Alternatively do

```bash
ipcontroller
ipengine --debug --log-to-file=True # start one engine with custom arguments
ipcluster engines --n=4 # start four more engines
```

Or alternatively do

```bash
ipython profile create --parallel --profile=myprofile # do this once
ipcluster --profile=myprofile # can also use profile with ipcontroller/ipengine commands
```
	
Make sure that Hoc can find the .hoc model files by either executing above command in the directory containing those files, or adding the relevant directories to the environment variable `$HOC_LIBRARY_PATH` (this could also be done in 
your protocol or cellmodel script using `os.environ["HOC_LIBRARY_PATH"]`)

## HOWTO: Optimization Parameters

Optimize parameters of reduced model. Passive parameters should be fitted first and set in the module file. Parameters to optimize, stimulation protocols to use for evaluation (objective function), and electrophysiological features extracted from the protocol responses can be set in following modules:

```python
optimize.bpop_parameters_stn
optimize.bpop_protocols_stn
optimize.bpop_features_stn
```

# Imports

Reduction method is Marasco method, 7 folding passes.

In [ ]:
%matplotlib notebook

# print date and time of script execution
import datetime
import pickle, pprint
pp = pprint.PrettyPrinter(indent=2)

# Third party
import bluepyopt.ephys as ephys

# Custom Modules
from bgcellmodels.models.STN.GilliesWillshaw import gillies_model
from bgcellmodels.models.STN.GilliesWillshaw.optimize import (
    bpop_cellmodels as stn_models,
    # bpop_optimize_stn as stn_opt
)

# Distributed logging
from bgcellmodels.common import logutils

logutils.setLogLevel('quiet',
    ['marasco', 'folding', 'redops', 'bluepyopt.ephys.parameters'])
# proto_logger = logging.getLogger('stn_protos')
# logutils.install_mp_handler(logger=proto_logger) # record some logs on multiple threads

In [ ]:
# print code version (hash of checked out version)
!git log -1 # --format="%H"

# print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
# %cd /home/luye/workspace/bgcellmodels/GilliesWillshaw/
# %pwd

In [ ]:
# OPTIMIZATION USING EXISTING SCRIPTS

# # Choose model we want to optimize
# cell_model = StnReducedModel(
#                 name        = 'StnFolded',
#                 fold_method = 'marasco',
#                 num_passes  = 7)

# # Maximum number of generations in optimization
# num_generations = 10

# # Checkpoints: for each generation save [population, generation, parents, halloffame, history, logbook, rndstate]
# checkpoints_file = '/home/luye/cloudstore_m/simdata/marasco_folding/opt_marasco7pass_checkpoints2.pkl'

# # Make optimisation
# optimisation, opt_data = make_optimisation(
#                             red_model = cell_model, 
#                             parallel = True,
#                             export_locals = False)

# Cell Model

Parameters of cell and morphology reduction

## Reduction Parameters

In [ ]:
# Choose model we want to optimize
fold_method, num_fold_passes = 'marasco', 7

red_model = stn_models.StnReducedModel(
                name        = 'StnFolded',
                fold_method = fold_method,
                num_passes  = num_fold_passes)

## Free/Frozen Parameters

In [ ]:
nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

# Make parameters
gleak_orig = 7.84112e-05 # Default in Gillies model
gleak_fit = 12.43169e-5 # fit to match Zin_DC (see praxis_passive.py)
dend_gl_param = ephys.parameters.NrnSectionParameter(
                    name		= 'gleak_dend_param',
                    param_name	= gillies_model.gleak_name,
                    locations	= [StnParameters.dendritic_region],
                    bounds		= [gleak_orig*1e-1, gleak_orig*1e1],
                    value		= gleak_fit, # SETPARAM: use fitted gl value
                    frozen		= True)

cm_orig = 1.0
cm_fit = cm_orig * (gleak_fit / gleak_orig) # preserve membrane time constant (praxis does not fit cm)
dend_cm_param = ephys.parameters.NrnSectionParameter(
                    name		= 'cm_dend_param',
                    param_name	= 'cm',
                    locations	= [StnParameters.dendritic_region],
                    bounds		= [cm_orig*1e-1, cm_orig*1e1],
                    value		= cm_fit, # SETPARAM: use fitted cm value
                    frozen		= True)

# FROZEN PARAMETERS are passive parameters fit previously in passive model
# NOTE: leave empty if you want to use passive params determined by folding method
frozen_params = [] # [dend_gl_param, dend_cm_param] # SETPARAM: frozen params from previous optimisations

# FREE PARAMETERS are active conductances with large impact on response
free_params = StnParameters.dend_active_params # SETPARAM: parameters that are optimised (must be not frozen)

# Make Optimization

## Stimulation Protocols

In [ ]:
stn_model_type = StnModel.Gillies_FoldMarasco

# Protocols to use for optimisation
opt_stim_protocols = [SYN_BACKGROUND_HIGH] # SETPARAM: stimulation protocols for evaluation of fitness

# Make all protocol data
red_protos = {
    stim_proto: BpopProtocolWrapper.make(stim_proto, stn_model_type) 
        for stim_proto in opt_stim_protocols
}

# Collect al frozen mechanisms and parameters required for protocols to work
proto_mechs, proto_params = BpopProtocolWrapper.all_mechs_params(red_protos.values())

# Distinguish between sets of parameters (used, frozen, free/optimised)
frozen_params += proto_params
used_params = frozen_params + free_params

for param in frozen_params:
    assert param.frozen
for param in free_params:
    assert (not param.frozen)

# Assign parameters to reduced model
red_model.set_mechs(proto_mechs)
red_model.set_params(used_params)

## Ephys Features

In [ ]:
# Get protocol responses for full model
if PROTO_RESPONSES_FILE is not None:
    full_responses = load_proto_responses(PROTO_RESPONSES_FILE)
else:
    full_protos = [BpopProtocolWrapper.make(stim_proto, stn_model_type) for stim_proto in opt_stim_protocols]
    full_mechs, full_params = BpopProtocolWrapper.all_mechs_params(full_protos)
    full_model = StnFullModel(
                    name		= 'StnGillies',
                    mechs		= full_mechs,
                    params		= full_params)
    full_responses = run_proto_responses(full_model, full_protos)

# Make EFEL feature objects
stimprotos_feats = StnFeatures.make_opt_features(red_protos.values())

# Calculate target values from full model responses
StnFeatures.calc_feature_targets(stimprotos_feats, full_responses)

## Cost Function (MOO objectives, fitness function)

In [ ]:
# Collect characteristic features for all protocols used in evaluation
all_opt_features, all_opt_weights = StnFeatures.all_features_weights(stimprotos_feats.values())

# # Make final objective function based on selected set of features
# total_objective = ephys.objectives.WeightedSumObjective(
#                                         name	= 'optimise_all',
#                                         features= all_opt_features,
#                                         weights	= all_opt_weights)
# all_objectives = [total_objective]

# ALTERNATIVE: set weights using 'exp_std'
all_objectives = [ephys.objectives.SingletonObjective(f.name, f) for f in all_opt_features]

# Calculator maps model responses to scores
fitcalc = ephys.objectivescalculators.ObjectivesCalculator(all_objectives)

# Make evaluator to evaluate model using objective calculator
opt_ephys_protos = {k.name: v.ephys_protocol for k,v in red_protos.iteritems()}
opt_params_names = [param.name for param in free_params]

cell_evaluator = ephys.evaluators.CellEvaluator(
                    cell_model			= red_model,
                    param_names			= opt_params_names, # fitted parameters
                    fitness_protocols	= opt_ephys_protos,
                    fitness_calculator	= fitcalc,
                    sim					= nrnsim,
                    isolate_protocols	= True)

## Evaluate Candidate 0

Evaluate initial model to adjust adjust scales (weight, standard error) of objectives

In [ ]:
used_param_names = [p.name for p in used_params]
default_params = {k:v for k,v in StnParameters.default_params.items() if k in used_param_names}

# Evaluate initial model
scores = cell_evaluator.evaluate_with_dicts(default_params)
pp.pprint(scores)

# NOTE: efeature objects are not copied, just references, so can change these
# NOTE: distance is sum(feat[i] - exp_mean) / N / exp_std  => so exp_std determines weight
for efeat, weight in zip(all_opt_features, all_opt_weights):
    logger.debug('Scaling EFeature {} : exp_std / weight = {} / {}'.format(efeat.name, scores[efeat.name], weight))
    efeat.exp_std = scores[efeat.name] / weight
    
# Verify: all scores schould be 1.0 * weight
# init_scores = cell_evaluator.evaluate_with_dicts(default_params)
# pp.pprint(init_scores)

# Optimize

In [ ]:
# Optimisation parameters
population_size = 50
num_generations = 15 # Maximum number of generations in optimization
parallel = True
deap_seed = 8 # SETPARAM: seed used by evolutionary algorithm (for population variability)

# Checkpoints: for each generation save [population, generation, parents, halloffame, history, logbook, rndstate]
import uuid
uuid_head = str(uuid.uuid1())[0:8]
checkpoints_file = '/home/luye/cloudstore_m/simdata/marasco_folding/opt_checkpoints_' + uuid_head + '.pkl'

# Make optimisation using the model evaluator
optimisation = bpop.optimisations.DEAPOptimisation(
                    evaluator		= cell_evaluator,
                    offspring_size	= population_size,
                    map_function = get_map_function(parallel),
                    seed = deap_seed)

In [ ]:
# Run optimisation
final_pop, hall_of_fame, logs, hist = optimisation.run(
                                        max_ngen = num_generations,
                                        cp_filename = checkpoints_file)

## Save optimisation data

In [ ]:
# Save dict {StimProtol: {feat_name : {exp_mean/std : value } } }
proto_feat_info = {}
for stim_proto, feat_dict in stimprotos_feats.iteritems():
    proto_feat_info[stim_proto.name] = {
        feat_name: {'weight': feat_data[1], 'exp_mean': feat_data[0].exp_mean, 'exp_std': feat_data[0].exp_std} 
            for feat_name, feat_data in feat_dict.iteritems()
    }

# Module info
import sys
code_version_info = {}
modulenames = set(sys.modules) & set(('bluepyopt', 'efel', 'elephant')) # & set(globals())
for module_name in modulenames:
    code_version_info[module_name] = getattr(sys.modules[module_name], '__version__', 'unknown')
head_SHA1 = %sx git log -1 --format="%H"
code_version_info['bgcellmodels'] = head_SHA1

In [ ]:
info_opt = {
    'opt_param_names': cell_evaluator.param_names, # in same order as individual params
    'free_params': {p.name: p.value for p in free_params},
    'frozen_params': {p.name: p.value for p in frozen_params},
    'deap_seed': deap_seed,
    'protos_feats': proto_feat_info,
    'code_version_info': code_version_info,
}

# pp.pprint(info_opt)
with open(checkpoints_file, 'ab') as f: # appends to file stream, load using second 'pickle.load()'
    pickle.dump(info_opt, f)

# Analyze optimization results

We want to know how our individuals (candidate parameter sets) evolved during the optimization, and potentially how their responses look.

In [ ]:
# OPTIONAL: reload previous results from checkpoints file
# old_checkpoints_file = '/home/luye/cloudstore_m/simdata/marasco_folding/opt_marasco7pass_checkpoints2.pkl'
# with open(old_checkpoints_file, 'r') as f:
#     checkpoint = pickle.load(f)
#     old_param_names = pickle.load(f)

# old_best_params = checkpoint['halloffame'][0]
# old_logs = checkpoint['logbook']

In [ ]:
# Import our analysis modules
from optimize.bpop_analysis_stn import plot_log

# Plot evolution of fitness values
plot_log(logs)

## Evaluate / Validate optimization results

We want to know how our optimized model performs under other stimulation protocols that were not part of the objective function. This is a form of cross-validation.

### Show final parameters

In [ ]:
from optimize.bpop_analysis_stn import (
    plot_responses,
    plot_fitness_scores,
    plot_diversity,
)
ephys_protocols = optimisation.evaluator.fitness_protocols

In [ ]:
# Print fittest individual (best parameter set)
best_params = optimisation.evaluator.param_dict(hall_of_fame[0])
print(pp.pprint(best_params))

### Compare final responses

In [ ]:
# Plot full model responses
full_responses = load_proto_responses(PROTO_RESPONSES_FILE)
full_resp = {}
for stimproto, responses in full_responses.iteritems():
    full_resp.update(responses)
plot_responses(full_resp)

In [ ]:
# Plot responses of best parameters
# best_responses = optimisation.evaluator.run_protocols(
#                     protocols    = ephys_protocols.values(),
#                     param_values = best_params)
# plot_responses(best_responses)

# Plot reduced model responses (to all available stimulation protocols)
model_type = StnModel.Gillies_FoldMarasco
all_stim_protos = StnProtocols.PROTOCOL_WRAPPERS.keys()
all_protos = [BpopProtocolWrapper.make(stim_proto, model_type) for stim_proto in all_stim_protos]
ephys_protos = [proto.ephys_protocol for proto in all_protos]
proto_mechs, proto_params = BpopProtocolWrapper.all_mechs_params(all_protos)

red_model = StnReducedModel( # make model version with all protocol mechanisms
                name        = 'StnFolded',
                fold_method = fold_method,
                num_passes  = num_fold_passes,
                mechs       = proto_mechs,
                params      = proto_params + free_params)
    
red_responses = {
    proto.name : proto.run(
                    cell_model		= red_model, 
                    param_values	= best_params,
                    sim				= nrnsim,
                    isolate			= True)
    for proto in ephys_protos
}

In [ ]:
plot_proto_responses(red_responses)

## Final fitness scores

In [ ]:
# Plot fitness (objective) scores
# objectives = optimisation.evaluator.fitness_calculator.calculate_scores(best_responses)
# plot_fitness_scores(objectives)

In [ ]:
# Plot diversity of parameters
plot_diversity(optimisation, checkpoints_file, optimisation.evaluator.param_names)
